In [ ]:
!pip install transformers

In [ ]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
datasets = load_dataset("squad_v2")

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [59]:
from transformers import pipeline
pipe = pipeline("question-answering", model="eragon0206/finetunedforQnA")

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [60]:
ctx = "My name is Ganesh and I am studying Data Science"
que = "What is Ganesh studying?"
pipe(context = ctx, question = que)

{'score': 0.0069444505497813225, 'start': 22, 'end': 23, 'answer': 'I'}

In [ ]:
datasets["validation"]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})

In [62]:
preds = []
acts = []

In [ ]:
for i in range(len(datasets["validation"])):
  preds.append(pipe(context = datasets["validation"]["context"][i], question = datasets["validation"]["question"][i])['answer'])
  acts.append(datasets["validation"]["answers"][i])

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(acts, preds, average='micro')

print(f"F1 Score: {f1}")


F1 Score: 0.5


In [ ]:
!pip install meteor-score


In [ ]:
from meteor import meteor_score


meteor = meteor_score(acts, preds)

print(f"METEOR Score: {meteor}")


In [ ]:
!pip install nltk


In [ ]:
for i in range(len(datasets["validation"])):
  preds.append[(pipe(context = datasets["validation"]["context"][i], question = datasets["validation"]["question"][i])['answer'])]
  acts.append(datasets["validation"]["answers"][i])

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

bleu_scores = [sentence_bleu(ref, pred.split()) for ref, pred in zip(acts, preds)]

average_bleu = sum(bleu_scores) / len(bleu_scores)

print(f"BLEU Scores: {bleu_scores}")
print(f"Average BLEU Score: {average_bleu}")


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.rouge_score import rouge_n, rouge_l

# Function to calculate ROUGE score
def calculate_rouge(reference, hypothesis):
    return {
        "rouge-1": rouge_n(reference, hypothesis, 1),
        "rouge-2": rouge_n(reference, hypothesis, 2),
        "rouge-l": rouge_l([reference], [hypothesis])
    }

# Calculate ROUGE scores for each pair
rouge_scores = [calculate_rouge(ref[0].split(), pred.split()) for ref, pred in zip(acts, preds)]

print(f"ROUGE Scores: {rouge_scores}")


In [ ]:
em_score = sum([1 if gt == pred else 0 for gt, pred in zip(acts, preds)]) / len(acts) * 100
print(f"Exact Match (EM) Score: {em_score}%")
